In [40]:
import pandas as pd
import kagglehub
team_stats = pd.read_csv('team_stats.csv')
path = kagglehub.dataset_download("philiphyde1/nfl-stats-1999-2022")
path = path.replace("\\", "/")  # For Windows compatibility
offense = pd.read_csv(f"{path}\\yearly_player_stats_offense.csv")
defense = pd.read_csv(f"{path}\\yearly_player_stats_defense.csv")


for df in [team_stats, offense, defense]:
    df['team'] = df['team'].str.strip().str.upper()
    df['season'] = df['season'].astype(int)


In [41]:
offense_filtered = offense[offense['position'].isin(['QB', 'RB', 'WR'])]

offense_summary = (
    offense_filtered.groupby(['season', 'team', 'position'])['fantasy_points_standard']
                    .sum()
                    .reset_index()
)

offense_pivot = offense_summary.pivot(index=['season', 'team'], columns='position', values='fantasy_points_standard')
offense_pivot = offense_pivot.reset_index().fillna(0)

# Ensure missing columns are added
for pos in ['QB', 'RB', 'WR']:
    if pos not in offense_pivot.columns:
        offense_pivot[pos] = 0

offense_pivot = offense_pivot.rename(columns={
    'QB': 'QB_Fantasy_Pts',
    'RB': 'RB_Fantasy_Pts',
    'WR': 'WR_Fantasy_Pts'
})


In [42]:
# defense['fantasy_points_calc'] = (
#     defense['solo_tackle'] * 1.5 +
#     defense['assist_tackle'] * 0.75 +
#     defense['sack'] * 4 +
#     defense['qb_hit'] * 1 +
#     defense['interception'] * 5 +
#     defense['fumble_forced'] * 3 +
#     defense['def_touchdown'] * 6 +
#     defense['safety'] * 2
# )
# defense_summary = (
#     defense.groupby(['season', 'team', 'position'])['fantasy_points_standard']
#                     .sum()
#                     .reset_index()
# )

# defense_filtered = defense[defense['position'].isin(['LB', 'CB', 'DT'])]


# defense_pivot = defense_filtered.pivot_table(
#     index=['season', 'team'],
#     columns='position',
#     values='fantasy_points_calc',
#     fill_value=0
# ).reset_index()
# defense_pivot.columns.name = None


# defense_pivot = defense_pivot.rename(columns={
#     'LB': 'LB_Fantasy_Pts',
#     'CB': 'CB_Fantasy_Pts',
#     'DT': 'DT_Fantasy_Pts'
# })


In [44]:
# Combine both fantasy dataframes
fantasy_summary = offense_pivot

# Merge with main team stats
merged = pd.merge(team_stats, fantasy_summary, on=['season', 'team'], how='left').fillna(0)

# Keep only the 6 fantasy columns + team stats
fantasy_cols = ['QB_Fantasy_Pts', 'RB_Fantasy_Pts', 'WR_Fantasy_Pts']
final_df = merged.copy()

# Add missing columns (if any)
for col in fantasy_cols:
    if col not in final_df.columns:
        final_df[col] = 0

# Save clean version
final_df.to_csv('team_stats_with_fantasy_clean.csv', index=False)
print("Cleaned file saved as 'team_stats_with_fantasy_clean.csv'")
print("Columns added:", [c for c in final_df.columns if 'Fantasy_Pts' in c])


Cleaned file saved as 'team_stats_with_fantasy_clean.csv'
Columns added: ['QB_Fantasy_Pts', 'RB_Fantasy_Pts', 'WR_Fantasy_Pts']


In [48]:
import requests
import pandas as pd
import numpy as np

season = 2025
positions = ["QB", "RB", "WR"]

# === FETCH PLAYER METADATA ===
print("Fetching player metadata...")
players_url = "https://api.sleeper.app/v1/players/nfl"
players_data = requests.get(players_url).json()

players_df = pd.DataFrame.from_dict(players_data, orient="index")
players_df = players_df[["player_id", "full_name", "team", "position"]]
players_df = players_df.dropna(subset=["position"])

# === FETCH SEASON STATS (TOTALS) ===
print("Fetching 2025 total stats...")
stats_url = f"https://api.sleeper.app/v1/stats/nfl/regular/{season}"
stats_data = requests.get(stats_url).json()

stats_df = pd.DataFrame.from_dict(stats_data, orient="index").reset_index()
stats_df.rename(columns={"index": "player_id"}, inplace=True)

# === FETCH PER-WEEK STATS TO DERIVE GAMES PLAYED ===
print("Fetching weekly stats to determine games played...")
weeks_data = []
for week in range(1, 19):  # up to week 18 (17 games + possible bye)
    week_url = f"https://api.sleeper.app/v1/stats/nfl/regular/{season}/{week}"
    r = requests.get(week_url)
    if r.status_code != 200:
        continue
    week_json = r.json()
    for pid, pdata in week_json.items():
        pdata["player_id"] = pid
        pdata["week"] = week
        weeks_data.append(pdata)

weekly_df = pd.DataFrame(weeks_data)
games_played = (
    weekly_df.groupby("player_id")["week"]
    .count()
    .reset_index()
    .rename(columns={"week": "games_played"})
)

# === MERGE ALL DATA ===
df = stats_df.merge(players_df, on="player_id", how="left").merge(games_played, on="player_id", how="left")
df = df[df["position"].isin(positions)]
df["games_played"] = df["games_played"].fillna(0)

# === COMPUTE PPR FANTASY POINTS ===
def calc_ppr(row):
    pass_pts = (row.get("pass_yd", 0) / 25) + (row.get("pass_td", 0) * 4) - (row.get("pass_int", 0) * 2)
    rush_pts = (row.get("rush_yd", 0) / 10) + (row.get("rush_td", 0) * 6)
    rec_pts = (row.get("rec_yd", 0) / 10) + (row.get("rec_td", 0) * 6) + row.get("rec", 0)
    fumbles = (row.get("fum_lost", 0) * -2)
    idp_pts = (row.get("idp_tkl", 0) * 1) + (row.get("idp_sack", 0) * 2) + \
              (row.get("idp_int", 0) * 3) + (row.get("idp_fum_rec", 0) * 2) + (row.get("idp_def_td", 0) * 6)
    return pass_pts + rush_pts + rec_pts + fumbles + idp_pts

df["pts_ppr"] = df.get("pts_ppr", np.nan)
df["pts_ppr"] = df["pts_ppr"].fillna(df.apply(calc_ppr, axis=1))

# === EXTRAPOLATE SEASON PROJECTION (assuming 17 games) ===
df["projected_full_season_pts"] = np.where(
    df["games_played"] > 0,
    df["pts_ppr"] / df["games_played"] * 17,
    0
)

# === OUTPUT CLEAN TABLE ===
df_out = df[["full_name", "team", "position", "games_played", "pts_ppr", "projected_full_season_pts"]]
df_out = df_out.sort_values("projected_full_season_pts", ascending=False)
print(df_out.head(20))

df_out.to_csv("fantasy_points_projected_2025.csv", index=False)
print("Saved to fantasy_points_projected_2025.csv")


Fetching player metadata...
Fetching 2025 total stats...
Fetching weekly stats to determine games played...
                full_name team position  games_played  pts_ppr  \
7367      Jonathan Taylor  IND       RB          10.0   273.90   
2615  Christian McCaffrey   SF       RB          10.0   248.80   
1315      Patrick Mahomes   KC       QB           9.0   211.46   
5803           Josh Allen  BUF       QB           9.0   209.66   
1579        Lamar Jackson  BAL       QB           6.0   136.56   
2492           Puka Nacua  LAR       WR           8.0   179.00   
648    Jaxon Smith-Njigba  SEA       WR           9.0   197.30   
859           Jalen Hurts  PHI       QB           8.0   174.78   
3333        De'Von Achane  MIA       RB          10.0   213.50   
7368           Drake Maye   NE       QB          10.0   212.70   
324      Matthew Stafford  LAR       QB           9.0   190.18   
1391       Justin Herbert  LAC       QB          10.0   210.80   
1335        Ja'Marr Chase  CIN    

In [47]:
stats_df.gms_active

0       NaN
1       NaN
2       NaN
3       NaN
4       3.0
       ... 
7555    NaN
7556    NaN
7557    NaN
7558    NaN
7559    NaN
Name: gms_active, Length: 7560, dtype: float64

In [49]:
import requests
import pandas as pd
import numpy as np

season = 2025
positions = ["QB", "RB", "WR"]

# === FETCH PLAYER METADATA ===
print("Fetching player metadata...")
players_url = "https://api.sleeper.app/v1/players/nfl"
players_data = requests.get(players_url).json()

players_df = pd.DataFrame.from_dict(players_data, orient="index")
players_df = players_df[["player_id", "team", "position"]]
players_df = players_df.dropna(subset=["position"])

# === FETCH SEASON STATS ===
print("Fetching 2025 stats...")
stats_url = f"https://api.sleeper.app/v1/stats/nfl/regular/{season}"
stats_data = requests.get(stats_url).json()

stats_df = pd.DataFrame.from_dict(stats_data, orient="index").reset_index()
stats_df.rename(columns={"index": "player_id"}, inplace=True)

# === FETCH PER-WEEK STATS TO DERIVE GAMES PLAYED ===
print("Fetching weekly stats to determine games played...")
weeks_data = []
for week in range(1, 19):  # 17 games + possible bye
    week_url = f"https://api.sleeper.app/v1/stats/nfl/regular/{season}/{week}"
    r = requests.get(week_url)
    if r.status_code != 200:
        continue
    week_json = r.json()
    for pid, pdata in week_json.items():
        pdata["player_id"] = pid
        pdata["week"] = week
        weeks_data.append(pdata)

weekly_df = pd.DataFrame(weeks_data)
games_played = (
    weekly_df.groupby("player_id")["week"]
    .count()
    .reset_index()
    .rename(columns={"week": "games_played"})
)

# === MERGE DATAFRAMES ===
df = stats_df.merge(players_df, on="player_id", how="left").merge(games_played, on="player_id", how="left")
df = df[df["position"].isin(positions)]
df["games_played"] = df["games_played"].fillna(0)

# === COMPUTE PPR FANTASY POINTS ===
def calc_ppr(row):
    pass_pts = (row.get("pass_yd", 0) / 25) + (row.get("pass_td", 0) * 4) - (row.get("pass_int", 0) * 2)
    rush_pts = (row.get("rush_yd", 0) / 10) + (row.get("rush_td", 0) * 6)
    rec_pts = (row.get("rec_yd", 0) / 10) + (row.get("rec_td", 0) * 6) + row.get("rec", 0)
    fumbles = (row.get("fum_lost", 0) * -2)
    return pass_pts + rush_pts + rec_pts + fumbles

df["pts_ppr"] = df.get("pts_ppr", np.nan)
df["pts_ppr"] = df["pts_ppr"].fillna(df.apply(calc_ppr, axis=1))

# === AGGREGATE TO TEAM LEVEL ===
team_pos_avg = (
    df.groupby(["team", "position"], as_index=False)
    .agg({"pts_ppr": "sum", "games_played": "sum"})
)

# === PIVOT SO EACH POSITION IS A COLUMN ===
team_wide = team_pos_avg.pivot(index="team", columns="position", values="pts_ppr").reset_index()
team_wide = team_wide.fillna(0)

# OPTIONAL: Rename for ML merge
team_wide = team_wide.rename(columns={
    "QB": "QB_Fantasy_Pts",
    "RB": "RB_Fantasy_Pts",
    "WR": "WR_Fantasy_Pts"
})

# Add season column
team_wide["season"] = season

# Reorder columns similar to training set
team_wide = team_wide[["team", "season", "QB_Fantasy_Pts", "RB_Fantasy_Pts", "WR_Fantasy_Pts"]]

print(team_wide.head())
team_wide.to_csv("team_fantasy_points_2025_ml.csv", index=False)
print("Saved to team_fantasy_points_2025_ml.csv")


Fetching player metadata...
Fetching 2025 stats...
Fetching weekly stats to determine games played...
position team  season  QB_Fantasy_Pts  RB_Fantasy_Pts  WR_Fantasy_Pts
0         ARI    2025          166.70           182.2           230.2
1         ATL    2025          123.66           253.6           216.9
2         BAL    2025          165.64           195.1           248.8
3         BUF    2025          211.92           203.5           287.9
4         CAR    2025          114.48           234.3           260.7
Saved to team_fantasy_points_2025_ml.csv


In [55]:
import pandas as pd

# Load CSVs
df1 = pd.read_csv("nfl_team_off_def_combined.csv")
df2 = pd.read_csv("team_fantasy_points_2025_ml.csv")

# Merge on 'season' and 'team'
merged_df = pd.merge(df1, df2, on=["season", "team"], how="left")  # or 'left', 'right', 'outer'

# Save result
merged_df.to_csv("merged_file.csv", index=False)
